### Cleaning refined and creating clean file v2
* &
* html syntax
* map (+ brand trfn; keep text -> EDA ) (save)
* clean (save)
* stem (save)
* lemma (save)
* gingerit (save)
* gingerit then lemma (save)
---

In [27]:
import numpy as np
import pandas as pd

In [28]:
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize
from nltk.stem import LancasterStemmer, PorterStemmer
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [30]:
import re

In [31]:
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
import plotly
import plotly.tools as tls   
import plotly.graph_objs as go

In [33]:
from gingerit.gingerit import GingerIt


In [34]:
import spacy
from spacy import displacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

In [35]:
import sys
sys.path.append("/Users/Aniket/MyStuff/Study/GreyAtom/Hackathon#3/Hack3_gen_functions")


In [36]:
import Hackathon3GeneralFunctions as genf

In [37]:
def map_iphone4(text):
    iphone4_pat = r"iphone[ ]+4"
    return re.sub(iphone4_pat, 'iPhone4', text, flags=re.I)

In [38]:
def map_ipads(text):
    #1
    ipads_pat = r"ipads"
    return re.sub(ipads_pat, 'ipad', text, flags=re.I)

In [39]:
def map_ipad2(text):
    #2
    ipad2_pat = r"ipad[ ]+2[s]?"
    return re.sub(ipad2_pat, 'ipad2', text, flags=re.I)

In [40]:
def map_brand(text):
    #EDA
    iphone_pat = r"iphone"
    ipad_pat = r"ipad"
    apple_pat = r"apple"
    temp = text
    temp = re.sub(iphone_pat, 'iPhone', temp, flags=re.I)
    temp = re.sub(ipad_pat, 'iPad', temp, flags=re.I)
    temp = re.sub(apple_pat, 'Apple', temp, flags=re.I)
    return temp

In [41]:
def replace_amp(text):
    pat_amp = r"&amp;"
    return re.sub(pat_amp, '&', text, flags=re.I)

In [42]:
def remove_amp(text):
    pat_amp_single = r" & "
    return re.sub(pat_amp_single, '', text, flags=re.I)
    

In [43]:
def remove_amp_op(text):
    pat_amp_single = r" & |& | &"
    return re.sub(pat_amp_single, '', text, flags=re.I)

In [44]:
def lemma(nlp, text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [45]:
def lemmatize_text(text):
    lnlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    return lemma(lnlp, text)

In [46]:
def remove_html_syn(text):    
#     html_syn_pat = r"&lt;[a-z]+&gt;"
    html_syntax_pat = r"&lt;[/a-z]+&gt;"
    return re.sub(html_syntax_pat, ' ', text, flags=re.I)

In [47]:
def replace_text_emoji(text):
    love_pat = r"&lt;3+"
    happy_pat = r":[-]?[\)]+"
    sad_pat = r":[-]?[\(]+"
    playful_pat = r":[-]?[p]+"
    wink_pat = r";[-]?[\)]+"
    straightface_pat = r":[-]?[\|]+"
    
    #replace text emojis
    temp = text
    temp =  re.sub(love_pat, ' love ', temp)
    temp =  re.sub(happy_pat, ' smiley ', temp)
    temp =  re.sub(sad_pat, ' sad ', temp)
    temp =  re.sub(playful_pat, ' playful ', temp, flags=re.I)
    temp =  re.sub(wink_pat, ' wink ', temp)
    temp =  re.sub(straightface_pat, ' straightface ', temp)
    return temp

In [48]:
def refine_before_cleaning(text):
    temp = text
    temp = map_iphone4(temp)
    temp = map_ipads(temp)
    temp = map_ipad2(temp)
    temp = remove_html_syn(temp)
    temp = replace_amp(temp)
    temp = remove_amp(temp)
    temp = replace_text_emoji(temp)
    return temp

In [49]:
# parser = GingerIt()
# line= "I loooveee Apple's new iPad ipad 2 with AT&T/GSM&D deal #ipad2"
# tweet=parser.parse(line)
# tweet

In [50]:
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

sentence = "The striped bats are hanging on their feet for best"

# Parse the sentence using the loaded 'en' model object `nlp`
doc = nlp(sentence)

# Extract the lemma for each token and join
" ".join([token.lemma_ for token in doc])
#> 'the strip bat be hang on -PRON- foot for good'

'the stripe bat be hang on -PRON- foot for good'

In [51]:
lemmatize_text("The striped bats are hanging on their feet for best #apple Apple")

'the stripe bat be hang on -PRON- foot for good # apple Apple'

In [52]:
remove_html_syn("&lt;title&gt;This is a title&lt;/title&gt;")

' This is a title '

In [53]:
refine_before_cleaning("&lt;3 ipad 2 iphone 4s &lt; &lt;title&gt; #ipad2")

' love  ipad2 iPhone4s &lt;   #ipad2'

In [54]:
train = pd.read_csv('data/train.csv', encoding='utf-8')
data = train.copy()

In [55]:
# data.loc[0:10,'tweet'].map(refine_before_cleaning)

In [56]:
data['tweet'] = data['tweet'].astype(str)

In [57]:
data['tweet_cleaned'] = data['tweet'].map(refine_before_cleaning)

In [59]:
data['tweet_cleaned'] = data['tweet_cleaned'].map(genf.clean_text)

In [58]:
data['tweet_cleaned'].head()

0    #sxswnui #sxsw #apple defining language of tou...
1    Learning ab Google doodles! All doodles should...
2    one of the most in-your-face ex. of stealing t...
3    This iPhone #SXSW app would b pretty awesome i...
4    Line outside the Apple store in Austin waiting...
Name: tweet_cleaned, dtype: object

In [60]:
data['tweet_cleaned'] = data['tweet_cleaned'].map(remove_amp_op)

In [61]:
data['tweet_cleaned'].head()

0     sxswnui  sxsw  apple defining language of tou...
1    learning ab google doodles  all doodles should...
2    one of the most in your face ex  of stealing t...
3    this iphone  sxsw app would b pretty awesome i...
4    line outside the apple store in austin waiting...
Name: tweet_cleaned, dtype: object

In [62]:
sent_raw = ' '.join(data['tweet'].astype(str))
sent_clean = ' '.join(data['tweet_cleaned'].astype(str))

In [63]:
# re.findall("."*8+"iphone[4]?"+"."*8,sent_clean)

In [64]:
data['tweet_stem'] = data['tweet_cleaned'].map(genf.tokenize_stem_join)

In [65]:
data['tweet_stem'].head()

0    sxswnui sxsw appl defin languag of touch with ...
1    learn ab googl doodl all doodl should be light...
2    one of the most in your face ex of steal the s...
3    thi iphon sxsw app would b pretti awesom if it...
4    line outsid the appl store in austin wait for ...
Name: tweet_stem, dtype: object

In [66]:
# data['tweet_lem'] = data['tweet_cleaned'].map(lemmatize_text)

In [67]:
# data['tweet_lem'].head()


In [68]:
# remove tw id 5025 and save

In [72]:
data.sentiment.value_counts(normalize=True)*100

1    59.265878
2    32.746769
0     6.268903
3     1.718449
Name: sentiment, dtype: float64

In [70]:
data.to_csv('data/train_cleaned_v2.csv', encoding='utf-8')

In [76]:
data_wo3 = data[data.sentiment != 3]

In [77]:
data_wo3.sentiment.value_counts(normalize=True)*100

1    60.302140
2    33.319345
0     6.378514
Name: sentiment, dtype: float64

In [78]:
data_wo3.to_csv('data/train_cleaned_v2_wo3.csv', encoding='utf-8')

In [79]:
#### test file

In [94]:
test = pd.read_csv('data/test.csv', encoding='utf-8')
test.shape

(1819, 2)

In [95]:
test['tweet'] = test['tweet'].astype(str)

In [96]:
test['tweet_cleaned'] = test['tweet'].map(refine_before_cleaning)

In [97]:
test['tweet_cleaned'] = test['tweet_cleaned'].map(genf.clean_text)

In [98]:
test['tweet_cleaned'].head()

0    audience q  what prototyping tools do you use ...
1    at sxsw  send your best photosvideos to       ...
2       and here  a pic of you winning your ipad   ...
3    google marissa mayer  mobile phone as a cursor...
4       sxsw google maps is even cooler than i thought
Name: tweet_cleaned, dtype: object

In [99]:
test['tweet_cleaned'] = test['tweet_cleaned'].map(remove_amp_op)

In [100]:
test['tweet_cleaned'].head()

0    audience q  what prototyping tools do you use ...
1    at sxsw  send your best photosvideos to       ...
2       and here  a pic of you winning your ipad   ...
3    google marissa mayer  mobile phone as a cursor...
4       sxsw google maps is even cooler than i thought
Name: tweet_cleaned, dtype: object

In [101]:
test_sent_raw = ' '.join(test['tweet'].astype(str))
test_sent_clean = ' '.join(test['tweet_cleaned'].astype(str))

In [102]:
# re.findall("."*8+"iphone[4]?"+"."*8,sent_clean)

In [103]:
test['tweet_stem'] = test['tweet_cleaned'].map(genf.tokenize_stem_join)

In [104]:
test.to_csv('data/test_cleaned_v2.csv', encoding='utf-8')